In [ ]:
import sys; sys.path.append("../")
from feature_extractor import FeatureExtractor
import torch
import numpy as np
import time
from cascade import CascadeClassifier
import cv2

In [ ]:
torch.cuda.is_available()

In [ ]:
parent = '../hFeatures5/'
data_path = 'new_data5'

# Features Extraction

In [ ]:
mean = 111.02404842061983
std = 56.926623499738575

In [ ]:
def transform1(img):
  return img
  # img = (img - img.mean()) / (img.std() + 1e-8)
  # return cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

In [ ]:

feature_extractor = FeatureExtractor(shape=(19,19),
                                     percentile=20,
                                     all_features_file=parent + "all_features.npz",
                                     selected_features_file=parent + "selected_features.npz",
                                     labels_file=parent + "labels.npy",
                                     indecies_file=parent + "indecies.npy",)

In [ ]:

X, y = feature_extractor.extractFeatures(pos_path='../'+data_path+'/train/face',
                                  neg_path='../'+data_path+'/train/non-face',
                                  transform=transform1,
                                  save_to_file=False)
print('done')


In [ ]:
# del feature_extractor.f2
# del feature_extractor.f3
# del feature_extractor.f4

In [ ]:
indecies, X = feature_extractor.selectPercentile(X, y )
# y = np.load(parent + 'labels.npy')
print(X.shape, y.shape)
print('done')


In [ ]:
# temp = np.load('../hFeatures3/all_features.npz')['arr_0']
temp = X
temp

In [ ]:
print(temp.min())
print(temp.max())
print(temp[3])
print(temp[4])

# Training

In [ ]:
[20]*10

In [ ]:
# adaboost from sklearn with base_estimator = DecisionTreeClassifier with max_depth = 1 (stump) and n_estimators = 200
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Take X.T because sklearn expects (n_samples, n_features)
X = X.T
y = y.T


print(X.shape, y.shape)

s_t = time.time()

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200)
clf.fit(X, y)

print('time:', time.time() - s_t)


In [ ]:
# %rm StrongClassifier/lastSC.last
# s_t = time.time()
# cascadeClassifier= CascadeClassifier(X, y, batchsize=5000, verbose=True, layers=[200])
# tr_acc = cascadeClassifier.train()

# cascadeClassifier.save(parent + 'cascadeClassifier.pkl')

# print('accuracy: ', tr_acc)
# print('time taken: ', time.time() - s_t, 's')
# print('done')

# Testing

In [ ]:
# accuracy imports
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [ ]:
X_test, y_test = feature_extractor.extractFeatures(pos_path='../'+data_path+'/test/face',
                                  neg_path='../'+data_path+'/test/non-face',
                                  transform=transform1,
                                  save_to_file=False)
print('done')


In [ ]:
X_test = X_test[indecies]

# convert from torch to numpy and take transpose
X_test = X_test.cpu().numpy().T
y_test = y_test.cpu().numpy()

print(X_test.shape, y_test.shape)
print('done')


In [ ]:
# Test accuracy
y_pred = clf.predict(X_test)


In [ ]:
# s_t = time.time()
# t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='../'+data_path+'/test/face',
#                                             neg_path='../'+data_path+'/test/non-face',
#                                             cascadeClassifier=cascadeClassifier,
#                                             transform=transform1)
# print(t_features.shape, t_labels.shape)
# predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
# print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
# print('time taken: ', time.time() - s_t, 's')

In [ ]:
# report
print('accuracy: ', accuracy_score(y_test, y_pred))
print('f1 score macro: ', f1_score(y_test, y_pred, average='macro'))
print('f1 score binary: ', f1_score(y_test, y_pred, average='binary'))
print('f1 score micro: ', f1_score(y_test, y_pred, average='micro'))
print('precision: ', precision_score(y_test, y_pred))
print('recall: ', recall_score(y_test, y_pred))
print('confusion matrix: \n', confusion_matrix(y_test, y_pred))
print('classification report: \n', classification_report(y_test, y_pred, digits=4))


# Try full image

In [ ]:
# !mkdir /content/drive/MyDrive/GP/GP_try/hFeatures5
# !cp -r ../hFeatures5/* /content/drive/MyDrive/GP/GP_try/hFeatures5

In [ ]:
!mkdir ../../lfw_all
!cp /content/drive/MyDrive/GP/lfw_all/* ../../lfw_all/

In [ ]:
import random
from detect_face import find_face
from PIL import Image
import os
import matplotlib.pyplot as plt
lfw_all = '../../lfw_all/'
all_images_names = os.listdir(lfw_all)
# all_images_names = sorted(all_images_names)
random.shuffle(all_images_names)
all_images_names = [lfw_all + name for name in all_images_names]
all_images_names[0]

In [ ]:
show_images = list(range(0, 20))
rectangles_only = True
# resize_to = (100, 100)
random.shuffle(all_images_names)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
feature_extractor.verbose = False

In [ ]:
for i in show_images:
  # >>>> Read image <<<<
  img = Image.open(all_images_names[i])
  print(img.mode)
  img = np.array(img)
  org_sh = img.shape
  if not rectangles_only:
    plt.imshow(img)
    plt.show()
  # get subwindows
  img2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  print(img2.shape)

  # >>>> Normalize image <<<<
  # normalize image
  # img2 = (img2 - img2.mean()) / img2.std()
  # img2 = cv2.resize(img, resize_to)
  # print(img2.shape)

  # img2 = cv2.normalize(img2, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
  # resize image
  # img2 = cv2.resize(img2, (570, 380))

  # show image
  if not rectangles_only:
    plt.imshow(img2, cmap='gray')
    plt.show()

  # +++++++++++++++++++++
  # find face
  # >>>> Find face <<<<
  face_coordinates, region_max_conf, max_conf , timing=\
                  find_face(img2, \
                              clf, \
                              feature_extractor, \
                              window_size=(19, 19), \
                              scale_dist=1.1, \
                              max_size=300, \
                              stride=5, \
                              device=device, \
                              verbose=False,
                              report_time=True,
                              use_sklearn=True)
  print(timing)
  # region_max_conf *= img.shape[0] / resize_to[0]
  # ++++++++++++++++++++
  # rectangles
  # >>>> Draw rectangles <<<<
  tmp_img = img.copy()
  for face_coordinate in face_coordinates:
      for x1, y1, x2, y2 in face_coordinate:
          cv2.rectangle(tmp_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
  # if not rectangles_only:
  plt.imshow(tmp_img)
  plt.show()

  # >>>> Draw rectangles with max confidence <<<<
  # amx confidence
  tmp_img = img.copy()
  x1, y1, x2, y2 = region_max_conf
  cv2.rectangle(tmp_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
  plt.imshow(tmp_img)
  plt.show()

